In [1]:
import os

In [2]:
%pwd

'/Users/susovanpatra/Desktop/E2E_ML_Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/susovanpatra/Desktop/E2E_ML_Project'

## Update the entity

In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

# This is the return type of Data Ingestion Config Customa Made
# This is ther entity

In [2]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

## Update Configuration Manager

In [3]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH) :
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion     # This is taking the custom datatype from config.yaml

        create_directories([config.root_dir])   # creating the root directory

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )   # Setting the type as DataIngestionConfig to return in that format

        return data_ingestion_config

## Update the components

In [5]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size
import pandas as pd


In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

## Update the pieline

In [28]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# The above part of the code is optional as I was getting SSL Certificate error,,So I disabled it (Not Recommended)

try:
    config = ConfigurationManager()                             
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


# 1. create the Configuration Manager [It will create the directory in artifacts and the directory of data ingestion]
# 2. get the DataIngestionConfig by calling the 'get_data_ingestion_config'
# 3. Now create an object of 'DataIngestion'
# 4. Inside the class we have functions - download_file & extract_zip_file. Call download_file
# 5. Now as we have downloaded it in zip format,,let's unzip the file

[2024-05-05 08:14:55,796]: INFO: common: yaml file: config/config.yaml loaded successfully:
[2024-05-05 08:14:55,802]: INFO: common: yaml file: params.yaml loaded successfully:
[2024-05-05 08:14:55,807]: INFO: common: yaml file: schema.yaml loaded successfully:
[2024-05-05 08:14:55,816]: INFO: common: created directory at: artifacts:
[2024-05-05 08:14:55,836]: INFO: common: created directory at: artifacts/data_ingestion:
[2024-05-05 08:15:32,902]: INFO: 2539110743: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 28FA:170316:2AC9CF:38B8E3:6636F2CA
Accept-Ranges: bytes
Date: S